In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /home/alberto/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/alberto/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:02<00:00, 5.44MB/s]


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [2]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)


In [3]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)


tensor([ 2.1052e+00, -1.7308e+00, -2.2278e+00, -2.9621e+00, -2.3761e+00,
         9.8064e-01, -1.5995e+00,  3.6929e+00,  6.3844e+00, -1.2970e+00,
        -6.7583e+00, -3.3542e+00, -7.9624e+00, -4.4576e+00, -5.6436e+00,
        -4.6615e+00, -1.9582e+00, -3.5506e-01, -1.2781e+00, -4.6722e+00,
        -3.2885e+00, -2.5688e+00, -2.4332e+00, -1.3037e+00, -3.2455e+00,
        -1.4179e+00, -1.1920e+00,  4.1882e-01, -1.6033e+00,  1.5931e+00,
         2.8403e-01, -6.2212e-01, -2.9329e-01, -3.8180e+00, -1.5392e+00,
        -2.8921e+00, -5.6117e-01, -2.3876e+00, -3.3341e-01,  1.2811e+00,
        -1.2503e+00, -2.6462e+00, -3.0978e+00, -2.2449e+00, -4.4174e-01,
        -1.2649e+00,  8.3433e-01, -2.0431e+00, -6.6184e-01, -8.3060e-02,
         4.9313e-01, -1.7198e+00, -7.7386e-01, -1.0981e+00, -5.3155e-01,
        -2.9183e+00, -1.9303e+00, -2.7204e+00, -6.0153e-01, -1.6766e+00,
         1.3525e+00, -4.2070e+00, -1.4710e+00, -4.5550e+00, -3.2666e+00,
        -4.0032e+00,  1.5984e-01, -1.9871e+00, -7.4

In [4]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


--2024-12-29 15:42:15--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.01s   

2024-12-29 15:42:15 (786 KB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [5]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())


Samoyed 0.8297895193099976
Pomeranian 0.07015100866556168
keeshond 0.012921061366796494
collie 0.010819987393915653
Great Pyrenees 0.009873730130493641
